## R3 To R1 Point Attractor, S2 Damping, R1 Box Avoidance

In [ ]:
using PBDS, StaticArrays, LinearAlgebra, GeometryBasics, Rotations, BenchmarkTools

## Setup

### Point Distance Attractor

In [ ]:
PBDS.metric_chart(xn, task::Attractor{<:DistanceFromPoint{R3,R1}}, CN::Chart{1,R1}) = 
    default_metric(xn, task, CN)
PBDS.potential_chart(xn, task::Attractor{<:DistanceFromPoint{R3,R1}}, CN::Chart{1,R1}) = xn[1]^2
PBDS.dissipative_forces_chart(xn, vn, task::Attractor{<:DistanceFromPoint{R3,R1}}, CN::Chart{1,R1}) =  -5*vn
PBDS.weight_metric_chart(xn, vn, task::Attractor{<:DistanceFromPoint{R3,R1}}, CN::Chart{1,R1}) = 
    default_weight_metric(xn, vn, task, CN)

### Angular Damping Around Goal

In [ ]:
PBDS.default_coord_rep(::Damping{<:AngularPositionAroundPoint{R3,S2}}) = EmbRep()
PBDS.metric_emb(xne, task::Damping{<:AngularPositionAroundPoint{R3,S2}}) =
    default_metric(xne, task)
PBDS.potential_emb(xne, task::Damping{<:AngularPositionAroundPoint{R3,S2}}) = 0.
PBDS.dissipative_forces_emb(xne, vne, task::Damping{<:AngularPositionAroundPoint{R3,S2}}) = -2*vne
PBDS.weight_metric_emb(xne, vne, task::Damping{<:AngularPositionAroundPoint{R3,S2}}) =
    default_weight_metric(xne, vne, task)

### Box Avoidance

In [ ]:
function PBDS.metric_chart(xn, task::CollisionAvoidance{<:DistanceSphereToBox{R3,R1}}, CN::Chart{1,R1})
    ψx = exp(1.e0 / xn[1]^2)
    G = SMatrix{1,1,eltype(xn)}([ψx])
end
PBDS.potential_chart(xn, task::CollisionAvoidance{<:DistanceSphereToBox{R3,R1}}, CN::Chart{1,R1}) = 0.
PBDS.dissipative_forces_chart(xn, vn, task::CollisionAvoidance{<:DistanceSphereToBox{R3,R1}}, CN::Chart{1,R1}) = 0*vn
function PBDS.weight_metric_chart(xn, vn, task::CollisionAvoidance{<:DistanceSphereToBox{R3,R1}}, CN::Chart{1,R1})
    offset_distance = 5.
    λ = (xn[1] < offset_distance && vn[1] < 0.) ? 1. : 0.
    W = SMatrix{1,1,eltype(xn)}(I)*λ
end

In [ ]:
xm_goal = SA[-6., 6., 0.]
M = R3
CM = Chart{1,M}()
tasks, CNs = TaskList(), ChartList()

N = R1
push!(tasks, Attractor(DistanceFromPoint{M,N}(xm_goal)))
push!(CNs, Chart{1,N}())

N = S2
push!(tasks, Damping(AngularPositionAroundPoint{M,N}(xm_goal)))
push!(CNs, Chart{SterProjSouth,N}())

N = R1
CN = Chart{1,N}()
radius = 0.5
center = SA[-3., 3., 0.]
rec_widths = SA[3., 3., 2.]
corner = @. center - rec_widths/2
rotation = RotXYZ(0.5, 0.5, 0.5)
push!(tasks, CollisionAvoidance(DistanceSphereToBox{M,N}(radius, Rect(Vec(corner), Vec(rec_widths)), rotation)))
push!(CNs, CN)

## Point Acceleration

In [ ]:
# Initial state
xm = SA[3., 4., 0.]
vm = SA[1., 1., 0.]
σxddot, = multiple_task_acceleration(xm, vm, tasks, CM, CNs)

## Single Trajectory Test

In [ ]:
Time = 10
dt = 0.01

traj = propagate_tasks(xm, vm, tasks, CM, CNs, Time, dt)
traj.xm[end]

## Multiple Trajectories

In [ ]:
using Makie, Observables, ProgressMeter, VectorizedRoutines

In [ ]:
Time = 13
dt = 0.03
xlim, Δx = 1., 2.
vlim, Δv = 2., 4.

xm0 = Array{Any}(undef, 3)
vm0 = Array{Any}(undef, 3)

xm0[1], xm0[2], xm0[3] = Matlab.meshgrid(-xlim:Δx:xlim, -xlim:Δx:xlim, -xlim:Δx:xlim)
vm0[1], vm0[2], vm0[3] = Matlab.meshgrid(-vlim:Δv:vlim, -vlim:Δv:vlim, -vlim:Δv:vlim)

for a in (xm0, vm0), i in 1:3
    a[i] = reshape(a[i], length(a[i]))
end

nx = length(xm0[1])
nv = length(vm0[1])
ix, iv = Matlab.meshgrid(1:nx, 1:nv)
ix = reshape(ix, length(ix))
iv = reshape(iv, length(iv))

ntraj = nx*nv
trajs = Array{Any}(undef, ntraj)
xm0 = [xm0[1]'; xm0[2]'; xm0[3]']
vm0 = [vm0[1]'; vm0[2]'; vm0[3]']
p = Progress(ntraj)

Threads.@threads for i in 1:ntraj
    trajs[i] = propagate_tasks(SA[xm0[:,ix[i]]...], SA[vm0[:,iv[i]]...], tasks, CM, CNs, Time, dt)
    isdefined(Main, :Test) || next!(p)
end

In [ ]:
fxx(i) = [trajs[j].xm[i][1] for j = 1:ntraj]
fxy(i) = [trajs[j].xm[i][2] for j = 1:ntraj]
fxz(i) = [trajs[j].xm[i][3] for j = 1:ntraj]
iobs = Observable(1)

ax_size, plot_size = 5, 800
limits = FRect3D((-ax_size-3, -ax_size+3, -ax_size), (2*ax_size, 2*ax_size, 2*ax_size))
scene = Scene(resolution = (plot_size, plot_size))
Makie.scatter!(scene, lift(i -> fxx(i), iobs), lift(i -> fxy(i), iobs), lift(i -> fxz(i), iobs), markersize = ax_size/20, color = :blue, limits = limits)
Makie.scatter!(scene, xm_goal', markersize = ax_size/20, color = :darkgreen)
rect = Rect(Vec(-rec_widths./2), Vec(rec_widths))
rect_mesh = mesh!(rect, color = :orange)
q = UnitQuaternion(rotation)
qrot = Quaternion(q.w, q.x, q.y, q.z)
Makie.rotate!(rect_mesh[end], qrot)
Makie.translate!(rect_mesh[end], center)
for i = 1:ntraj
    Makie.lines!(scene, getindex.(trajs[i].xm,1), getindex.(trajs[i].xm,2), getindex.(trajs[i].xm,3), color = :purple)
end

Makie.xlabel!(scene, "x")
Makie.ylabel!(scene, "y")
axis = scene[Axis]
axis.showaxis = false
display(scene)

In [ ]:
function record_scene(scene, filename, iobs, N, framerate=60)
    p = Progress(N)
    record(scene, filename, 1:N) do i
        iobs[] = i
        rotate_cam!(scene, 0.01, 0., 0.)
        isdefined(Main, :Test) || next!(p)
    end
    isdefined(Main, :Test) || display("text/html", html_video(filename))
end

filename = "R3ToR1PointDistanceAttractor_S2Damping_R1BoxAvoidance.mp4"
record_scene(scene, filename, iobs, length(trajs[1].xm))